In [1]:
# !wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [2]:
# !unzip uncased_L-12_H-768_A-12.zip

In [3]:
!rm -rf data/.ipynb_checkpoints

In [4]:
from utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
import random
import os

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [5]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [6]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

In [7]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

W0728 12:08:01.512709 139862809098048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [8]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

W0728 12:08:01.526020 139862809098048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [9]:
MAX_SEQ_LENGTH = 100

In [10]:
from tqdm import tqdm

input_ids, input_masks, segment_ids = [], [], []

for text in tqdm(trainset.data):
    tokens_a = tokenizer.tokenize(text)
    if len(tokens_a) > MAX_SEQ_LENGTH - 2:
        tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_id = [0] * len(tokens)
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    padding = [0] * (MAX_SEQ_LENGTH - len(input_id))
    input_id += padding
    input_mask += padding
    segment_id += padding
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 10662/10662 [00:02<00:00, 4605.25it/s]


In [11]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [12]:
tokenizer.tokenize(trainset.data[0])

['worse', 'film', 'breaking', 'breaking', 'utterly', 'charming']

In [13]:
epoch = 10
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(input_ids) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [14]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=True,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        self.logits = tf.layers.dense(output_layer, dimension_output)
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

W0728 12:08:04.883535 139862809098048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0728 12:08:04.888481 139862809098048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:409: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0728 12:08:04.932405 139862809098048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

W0728 12:08:05.471327 139862809098048 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (

In [16]:
def extract_attention_weights(tf_graph):
    num_layers = bert_config.num_hidden_layers
    attns = [{'layer_%s' % i: tf_graph.get_tensor_by_name('bert/encoder/layer_%s/attention/self/Softmax:0' % i)}
             for i in range(num_layers)]

    return attns

In [17]:
attns = extract_attention_weights(tf.get_default_graph())
model.attns = attns

In [18]:
sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)

W0728 12:08:20.580060 139862809098048 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [19]:
train_input_ids, test_input_ids, train_input_masks, test_input_masks, train_segment_ids, test_segment_ids, train_Y, test_Y = train_test_split(
    input_ids, input_masks, segment_ids, trainset.target, test_size = 0.2
)

In [20]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_masks = train_input_masks[i: index]
        batch_segment = train_segment_ids[i: index]
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_masks = test_input_masks[i: index]
        batch_segment = test_segment_ids[i: index]
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_input_ids) / batch_size
    train_acc /= len(train_input_ids) / batch_size
    test_loss /= len(test_input_ids) / batch_size
    test_acc /= len(test_input_ids) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/143 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.813835
time taken: 204.77616906166077
epoch: 0, training loss: 0.546556, training acc: 0.722046, valid loss: 0.449890, valid acc: 0.813835



train minibatch loop:   0%|          | 0/143 [00:00<?, ?it/s]

time taken: 200.3038547039032
epoch: 1, training loss: 0.365002, training acc: 0.846914, valid loss: 0.483403, valid acc: 0.810169



train minibatch loop:   0%|          | 0/143 [00:00<?, ?it/s]

time taken: 200.25593400001526
epoch: 2, training loss: 0.201564, training acc: 0.931176, valid loss: 0.805201, valid acc: 0.776584



test minibatch loop: 100%|██████████| 36/36 [00:17<00:00,  2.40it/s, accuracy=0.727, cost=1.48] 

time taken: 200.3201220035553
epoch: 3, training loss: 0.116890, training acc: 0.966702, valid loss: 0.900214, valid acc: 0.802924

break epoch:4



In [67]:
batch_x = [input_ids[-1]]
batch_segment = [segment_ids[-1]]
batch_masks = [input_masks[-1]]
trainset.data[-1]

'time plot grinds increasingly incoherent fashion might wishing watch makes time go faster rather way around'

In [68]:
attentions = sess.run(model.attns, feed_dict = {model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks})

In [69]:
cls_attn = attentions[0]['layer_0'][:, :, 0, :]
cls_attn = np.mean(cls_attn, axis=1)

In [70]:
total_weights = np.sum(cls_attn, axis=-1, keepdims=True)
attn = (cls_attn / total_weights)[0]
attn.shape

(100,)

In [71]:
def merge_wordpiece_tokens(paired_tokens):
    """
    Combine tokens that have been broken up during the wordpiece tokenization process
    The weights of the merged token is the average of their wordpieces
    :param paired_tokens: A list of tuples with the form (wordpiece_token, attention_weight)
    :return: A list of tuples with the form (word_token, attention_weight)
    """
    new_paired_tokens = []
    n_tokens = len(paired_tokens)

    i = 0

    while i < n_tokens:
        current_token, current_weight = paired_tokens[i]
        if current_token.startswith('##'):
            previous_token, previous_weight = new_paired_tokens.pop()
            merged_token = previous_token
            merged_weight = [previous_weight]
            while current_token.startswith('##'):
                merged_token = merged_token + current_token.replace('##', '')
                merged_weight.append(current_weight)
                i = i + 1
                current_token, current_weight = paired_tokens[i]
            merged_weight = np.mean(merged_weight)
            new_paired_tokens.append((merged_token, merged_weight))

        else:
            new_paired_tokens.append((current_token, current_weight))
            i = i + 1
    
    words = [i[0] for i in new_paired_tokens if i[0] not in ['[CLS]', '[SEP]', '[PAD]']]
    weights = [i[1] for i in new_paired_tokens if i[0] not in ['[CLS]', '[SEP]', '[PAD]']]
    weights = np.array(weights)
    weights = weights / np.sum(weights)
    return list(zip(words, weights))

In [72]:
merge_wordpiece_tokens(list(zip(tokenizer.convert_ids_to_tokens(batch_x[0]), attn)))

[('time', 0.07921354),
 ('plot', 0.05070206),
 ('grinds', 0.073813565),
 ('increasingly', 0.057528302),
 ('incoherent', 0.06072644),
 ('fashion', 0.07929965),
 ('might', 0.039253872),
 ('wishing', 0.06646434),
 ('watch', 0.04416551),
 ('makes', 0.038636986),
 ('time', 0.09167157),
 ('go', 0.09118808),
 ('faster', 0.051011782),
 ('rather', 0.055073477),
 ('way', 0.06224899),
 ('around', 0.05900185)]

In [73]:
cls_attn = attentions[-1]['layer_11'][:, :, 0, :]
cls_attn = np.mean(cls_attn, axis=1)
total_weights = np.sum(cls_attn, axis=-1, keepdims=True)
attn = (cls_attn / total_weights)[0]
attn.shape

(100,)

In [74]:
merge_wordpiece_tokens(list(zip(tokenizer.convert_ids_to_tokens(batch_x[0]), attn)))

[('time', 0.056819078),
 ('plot', 0.0632532),
 ('grinds', 0.06938076),
 ('increasingly', 0.08377443),
 ('incoherent', 0.024481593),
 ('fashion', 0.02720975),
 ('might', 0.026110101),
 ('wishing', 0.015731115),
 ('watch', 0.08168129),
 ('makes', 0.08993647),
 ('time', 0.09198012),
 ('go', 0.029810423),
 ('faster', 0.1186272),
 ('rather', 0.05931811),
 ('way', 0.05009796),
 ('around', 0.111788414)]

In [75]:
combined_attentions = []
for a in attentions:
    combined_attentions.append(list(a.values())[0])
    
cls_attn = np.mean(combined_attentions, axis = 0).mean(axis = 2)
cls_attn = np.mean(cls_attn, axis=1)
total_weights = np.sum(cls_attn, axis=-1, keepdims=True)
attn = (cls_attn / total_weights)[0]
attn.shape

(100,)

In [76]:
merge_wordpiece_tokens(list(zip(tokenizer.convert_ids_to_tokens(batch_x[0]), attn)))

[('time', 0.053206075),
 ('plot', 0.058655694),
 ('grinds', 0.058348235),
 ('increasingly', 0.056193475),
 ('incoherent', 0.04362384),
 ('fashion', 0.0641345),
 ('might', 0.073644035),
 ('wishing', 0.07437738),
 ('watch', 0.06467828),
 ('makes', 0.07696252),
 ('time', 0.054407526),
 ('go', 0.04876398),
 ('faster', 0.07310601),
 ('rather', 0.07628402),
 ('way', 0.045983776),
 ('around', 0.07763063)]